(optional)

In [16]:
# from google.colab import drive
# import os
# drive.mount('/content/drive')

In [17]:
# os.chdir('...')

# **HW5: Brain signal classification**
In *HW 5*, you need to finish:

1.  Model Implementation Part: Implement LSTM and EEGNet models to predict the label of each samples.

2.  Model Competition Part: Implementing a model to reach better accuracy performance.

In [18]:
import numpy as np
import os
import math
import csv
import matplotlib.pyplot as plt
# Import the packages you need here

import tensorflow as tf
from tensorflow.keras import models, layers

In [19]:
data = np.load('data.npz')
label = np.load('label.npz')

In [20]:
X_train = data['X_train']
X_val = data['X_val']
X_test = data['X_test']

Y_train = label['Y_train']
Y_val = label['Y_val']

In [21]:
X_train.shape, X_val.shape, X_test.shape

((604, 22, 200), (152, 22, 200), (190, 22, 200))

In [22]:
Y_train.shape, Y_val.shape

((604, 1), (152, 1))

## Model Implementation Part

### LSTM

In [52]:
# Build your model here:
model_LSTM = models.Sequential()
model_LSTM.add(layers.InputLayer(input_shape=(22, 200)))
model_LSTM.add(layers.LSTM(64, activation='relu', return_sequences=True, dropout=0.2))
model_LSTM.add(layers.LSTM(32, activation='sigmoid', return_sequences=False))
model_LSTM.add(layers.Dense(6))
model_LSTM.add(layers.Activation('softmax', name = 'softmax'))
model_LSTM.compile(optimizer = 'adam', loss='sparse_categorical_crossentropy', metrics = ['accuracy'])
model_LSTM.summary()
history = model_LSTM.fit(X_train, Y_train, steps_per_epoch=12, validation_data=(X_val, Y_val), validation_steps=4, epochs=20)

Model: "sequential_38"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_48 (LSTM)              (None, 22, 64)            67840     
                                                                 
 lstm_49 (LSTM)              (None, 32)                12416     
                                                                 
 dense_27 (Dense)            (None, 6)                 198       
                                                                 
 softmax (Activation)        (None, 6)                 0         
                                                                 
Total params: 80,454
Trainable params: 80,454
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
12/12 [==============================] - 3s 52ms/step - loss: 1.8620 - accuracy: 0.1573 - val_loss: 1.8389 - val_accuracy: 0.1579
Epoch 2/20
12/12 [======================

In [ ]:
output = "..."
assert(output.shape == (190, 1))
np.savetxt('lstm_output.csv', output, delimiter=",")

### EEGNet

In [53]:
# Build your model here:
def EEGNet(classes = 6, channels = 22, timeframe = 200, droprate = 0.5, filter1 = 8, depth = 8, filter2 = 16, normrate = 0.25):
    model = models.Sequential()
    model.add(layers.InputLayer(input_shape=(channels, timeframe, 1)))
    model.add(layers.Conv2D(filter1, (1, 64), padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.DepthwiseConv2D((channels, 1), use_bias = False, depth_multiplier = depth, depthwise_constraint = tf.keras.constraints.max_norm(1.)))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('elu'))
    model.add(layers.AveragePooling2D((1, 4)))
    model.add(layers.Dropout(droprate))
    model.add(layers.SeparableConv2D(filter2, (1, 16), use_bias = False, padding = 'same'))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('elu'))
    model.add(layers.AveragePooling2D((1, 8)))
    model.add(layers.Dropout(droprate))
    model.add(layers.Flatten(name = 'flatten'))
    model.add(layers.Dense(classes, name = 'dense', kernel_constraint = tf.keras.constraints.max_norm(normrate)))
    model.add(layers.Activation('softmax', name = 'softmax'))
    return model
model_EEGNet = EEGNet()
model_EEGNet.compile(optimizer = 'adam', loss='sparse_categorical_crossentropy', metrics = ['accuracy'])
model_EEGNet.summary()
history = model_EEGNet.fit(X_train, Y_train, validation_data=(X_val, Y_val), epochs=20)

Model: "sequential_39"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 22, 200, 8)        512       
                                                                 
 batch_normalization_9 (Batc  (None, 22, 200, 8)       32        
 hNormalization)                                                 
                                                                 
 depthwise_conv2d_3 (Depthwi  (None, 1, 200, 64)       1408      
 seConv2D)                                                       
                                                                 
 batch_normalization_10 (Bat  (None, 1, 200, 64)       256       
 chNormalization)                                                
                                                                 
 activation_6 (Activation)   (None, 1, 200, 64)        0         
                                                     

In [ ]:
output = "..."
assert(output.shape == (190, 1))
np.savetxt('eegnet_output.csv', output, delimiter=",")

## Model Competition Part

In [ ]:
# Build your model here:


In [ ]:
output = "..."
assert(output.shape == (190, 1))
np.savetxt('competition_output.csv', output, delimiter=",")